# Train Mountain Car

[OpenAI Gym](http://gym.openai.com) has been designed in such a way that all environments provide the same API - i.e. the same methods `reset`, `step` and `render`, and the same abstractions of **action space** and **observation space**. Thus is should be possible to adapt the same reinforcement learning algorithms to different environments with minimal code changes.

## A Mountain Car Environment

[Mountain Car environment](https://gym.openai.com/envs/MountainCar-v0/) contains a car stuck in a valley:

The goal is to get out of the valley and capture the flag, by doing at each step one of the following actions:

| Value | Meaning |
|---|---|
| 0 | Accelerate to the left |
| 1 | Do not accelerate |
| 2 | Accelerate to the right |

The main trick of this problem is, however, that the car's engine is not strong enough to scale the mountain in a single pass. Therefore, the only way to succeed is to drive back and forth to build up momentum.

Observation space consists of just two values:

| Num | Observation  | Min | Max |
|-----|--------------|-----|-----|
|  0  | Car Position | -1.2| 0.6 |
|  1  | Car Velocity | -0.07 | 0.07 |

Reward system for the mountain car is rather tricky:

 * Reward of 0 is awarded if the agent reached the flag (position = 0.5) on top of the mountain.
 * Reward of -1 is awarded if the position of the agent is less than 0.5.

Episode terminates if the car position is more than 0.5, or episode length is greater than 200.
## Instructions

Adapt our reinforcement learning algorithm to solve the mountain car problem. Start with existing [notebook.ipynb](notebook.ipynb) code, substitute new environment, change state discretization functions, and try to make existing algorithm to train with minimal code modifications. Optimize the result by adjusting hyperparameters.

> **Note**: Hyperparameters adjustment is likely to be needed to make algorithm converge. 


Solution

In [25]:
import sys
import gymnasium as gym
import matplotlib.pyplot as plt
import numpy as np
import random

Create a mountain car environment

In [26]:
env = gym.make("MountainCar-v0",render_mode = "human")
print(env.action_space)
print(env.observation_space)
print(env.action_space.sample())

Discrete(3)
Box([-1.2  -0.07], [0.6  0.07], (2,), float32)
1


To see how the environment works, let's run a short simulation for 100 steps. At each step, we provide one of the actions to be taken - in this simulation we just randomly select an action from action_space.

In [27]:
env = gym.make("MountainCar-v0",render_mode = "human")
observation, info = env.reset()

for i in range(100):
   action = env.action_space.sample()  # agent policy that uses the observation and info
   observation, reward, terminated, truncated, info = env.step(action)
   #if terminated or truncated:
     # observation, info = env.reset()
env.close()

During simulation, we need to get observations in order to decide how to act. In fact, step function returns us back current observations, reward function, and the done flag that indicates whether it makes sense to continue the simulation or not:

In [28]:
env = gym.make("MountainCar-v0", render_mode = "human")
observation, info = env.reset()
terminated = False
truncated = False

while not terminated and not truncated:
    action = env.action_space.sample()  # agent policy that uses the observation and info
    observation, reward, terminated, truncated, info = env.step(action)
    print(f"{observation} -> {reward}")
   
env.close()

[-0.5389757  0.0011237] -> -1.0
[-0.5367367   0.00223899] -> -1.0
[-0.53539926  0.0013375 ] -> -1.0
[-5.3497326e-01  4.2598482e-04] -> -1.0
[-5.3546196e-01 -4.8872316e-04] -> -1.0
[-0.5368617  -0.00139977] -> -1.0
[-0.53816205 -0.00130032] -> -1.0
[-0.5393532  -0.00119113] -> -1.0
[-0.54142624 -0.00207302] -> -1.0
[-0.5433656  -0.00193938] -> -1.0
[-0.5461568  -0.00279121] -> -1.0
[-0.54877895 -0.00262216] -> -1.0
[-0.5522124  -0.00343348] -> -1.0
[-0.5554316  -0.00321914] -> -1.0
[-0.5584124  -0.00298076] -> -1.0
[-0.5601325  -0.00172013] -> -1.0
[-5.605792e-01 -4.466793e-04] -> -1.0
[-5.6074905e-01 -1.6989499e-04] -> -1.0
[-0.5596409   0.00110816] -> -1.0
[-0.55726296  0.00237795] -> -1.0
[-0.55463296  0.00263   ] -> -1.0
[-0.5507705   0.00386242] -> -1.0
[-0.5477046   0.00306598] -> -1.0
[-0.544458    0.00324662] -> -1.0
[-0.54105496  0.00340296] -> -1.0
[-0.5375212   0.00353382] -> -1.0
[-0.53488296  0.00263821] -> -1.0
[-0.53216016  0.00272282] -> -1.0
[-0.5293731   0.00278703] ->

We can get min and max value of those numbers:

In [29]:
print(env.observation_space.low)
print(env.observation_space.high)

[-1.2  -0.07]
[0.6  0.07]


In [30]:
def discretize(x):
    return tuple(((x - np.array([-1.2, -0.07])) / np.array([0.1, 0.01])).astype(int))

In [35]:
env = gym.make("MountainCar-v0", render_mode = "human")
observation, info = env.reset()
terminated = False
truncated = False

while not terminated and not truncated:
    action = env.action_space.sample()  # agent policy that uses the observation and info
    observation, reward, terminated, truncated, info = env.step(action)
    print(discretize(observation))

env.close()

(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 7)
(6, 8)
(7, 8)
(7, 8)
(7, 8)
(7, 7)
(7, 7)
(7, 7)
(7, 7)
(7, 7)
(7, 7)
(7, 7)
(7, 7)
(7, 7)
(7, 7)
(7, 7)
(7, 7)
(7, 7)
(7, 7)
(7, 7)
(7, 6)
(7, 6)
(7, 6)
(7, 6)
(7, 6)
(7, 6)
(7, 6)
(7, 6)
(7, 6)
(7, 6)
(7, 6)
(7, 6)
(7, 6)
(7, 6)
(7, 6)
(7, 6)
(6, 6)
(6, 5)
(6, 6)
(6, 5)
(6, 6)
(6, 5)
(6, 5)
(6, 5)
(6, 5)
(6, 6)
(5, 6)
(5, 6)
(5, 6)
(5, 6)
(5, 6)
(5, 6)
(5, 6)
(5, 6)
(5, 6)
(5, 6)
(5, 6)
(5, 6)
(5, 6)
(5, 6)
(5, 7)
(5, 7)
(5, 7)
(5, 7)
(5, 7)
(5, 7)
(5, 7)
(5, 7)
(5, 7)
(5, 7)
(6, 7)
(6, 8)
(6, 7)
(6, 8)
(6, 8)
(6, 8)
(6, 8)
(6, 7)
(6, 7)
(6, 7)
(7, 7)
(7, 7)
(7, 7)
(7, 7)
(7, 7)
(7, 7)
(7, 8)
(7, 7)
(7, 7)
(7, 7)
(7, 7)
(8, 7)
(8, 7)
(8, 7)
(8, 7)
(8, 7)
(8, 7)
(8, 6)
(8, 6)
(8, 6)
(8, 6)
(8, 6)
(8, 6)
(7, 6)
(7, 6)
(7, 6)
(7, 6)
(7, 5)
(7, 5)
(7, 5)
(7, 5)
(7, 5)
(6, 5)
(6, 5)
(6, 5)
(6, 5)
(6, 5)
(6, 5)
(6, 5)
(6, 5)
(6, 5)
(5, 6)
(5, 6)
(5, 6)
(5, 6)
(5, 6)
(5, 6)
(5, 6)
(5, 6)
(5, 6)
(5, 6)
(5, 6)

In [39]:
Q = {}
actions = (0,1,2)

def qvalues(state):
    return [Q.get((state,a),0) for a in actions]

In [40]:
# hyperparameters
alpha = 0.3
gamma = 0.9
epsilon = 0.90

Training is computationally expensive. So, slow and steady. 

In [ ]:
def probs(v,eps=1e-4):
    v = v-v.min()+eps
    v = v/v.sum()
    return v

env = gym.make("MountainCar-v0", render_mode = "human")
Qmax = 0
cum_rewards = []
rewards = []
for epoch in range(1000):
    observation, info = env.reset()
    cum_reward=0
    terminated = False
    truncated = False
    # == do the simulation ==
    while not terminated and not truncated:
        s = discretize(observation)
        if random.random()<epsilon:
        # exploitation - chose the action according to Q-Table probabilities
            v = probs(np.array(qvalues(s)))
            a = random.choices(actions,weights=v)[0]
        else:
        # exploration - randomly chose the action
            a = np.random.randint(env.action_space.n)

        observation, reward, terminated, truncated, info = env.step(a)
        cum_reward+=reward
        ns = discretize(observation)
        Q[(s,a)] = (1 - alpha) * Q.get((s,a),0) + alpha * (reward + gamma * max(qvalues(ns)))
    
                
    cum_rewards.append(cum_reward)
    rewards.append(cum_reward)
    # == Periodically print results and calculate average reward ==
    if epoch%50==0:
        print(f"{epoch}: {np.average(cum_rewards)}, alpha={alpha}, epsilon={epsilon}")
        if np.average(cum_rewards) > Qmax:
            Qmax = np.average(cum_rewards)
            Qbest = Q
        cum_rewards=[]

In [ ]:
plt.plot(rewards)

In [ ]:
def running_average(x,window):
    return np.convolve(x,np.ones(window)/window,mode='valid')

plt.plot(running_average(rewards,100))

In [45]:
env = gym.make("MountainCar-v0", render_mode = "human")
observation, info = env.reset()
terminated = False
truncated = False
while not terminated and not truncated:
    s = discretize(observation)
    v = probs(np.array(qvalues(s)))
    a = random.choices(actions,weights=v)[0]
    observation, reward, terminated, truncated, info = env.step(a)
    
env.close()

# Saving result to an animated GIF
If you want to impress your friends, you may want to send them the animated GIF picture of the mountain car. To do this, we can invoke env.render to produce an image frame, and then save those to animated GIF using PIL library:

In [48]:
from PIL import Image
env = gym.make("MountainCar-v0", render_mode = "rgb_array")
observation, info = env.reset()
i=0
ims = []
terminated = False
truncated = False
while not terminated and not truncated:
   s = discretize(observation)
   img= env.render()
   ims.append(Image.fromarray(img))
   v = probs(np.array([Q.get((s,a),0) for a in actions]))
   a = random.choices(actions,weights=v)[0]
   observation, reward, terminated, truncated, info = env.step(a)
   i+=1
   
env.close()
ims[0].save('images/mountain-car.gif',save_all=True,append_images=ims[1::2],loop=0,duration=5)
print(i)

200
